In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler

dataset=pd.read_csv("train_set.tsv", sep='\t', header=0)
dataset_numpy=dataset.to_numpy()
x=dataset_numpy[:,:-2]
y=dataset_numpy[:,-2:]



In [3]:

scaler = MinMaxScaler()
# scaler = StandardScaler()
scaler.fit(x)
x=scaler.transform(x)





In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(750, 35) (750, 2) (250, 35) (250, 2)


In [ ]:
#if you want to undersample the data doenst work because i've not too much data 
from imblearn.under_sampling import RandomUnderSampler
over_sampler = RandomUnderSampler(random_state=42)
x_train, y_train = over_sampler.fit_resample(x_train, y_train[:,0])
y_train=y_train.reshape(-1,1)

In [ ]:
#if you want to oversample i dont think it's a good idea because i've not too much data 
from imblearn.over_sampling import RandomOverSampler
over_sampler = RandomOverSampler(random_state=42)
x_train, y_train = over_sampler.fit_resample(x_train, y_train[:,0])
y_train=y_train.reshape(-1,1)


In [ ]:
#evaluate label weights
from collections import Counter
c=Counter(y_train[:,0])
for k,v in c.items():
    c[k]=v/len(y_train)
print(c)

In [7]:
def classificationResults(x_train,x_test,y_train,y_test,model):
    if y_train.shape[1]>1:
        model.fit(x_train,y_train[:,0])
        y_pred_train=model.predict(x_train)
        y_pred_test=model.predict(x_test)
        print("Train F1 score: ",f1_score(y_train[:,0],y_pred_train,average='micro'))
        print("Train Precision score: ",precision_score(y_train[:,0],y_pred_train,average='micro'))
        print("Train Recall score: ",recall_score(y_train[:,0],y_pred_train,average='micro'))
        print("Test F1 score: ",f1_score(y_test[:,0],y_pred_test,average='micro'))
        print("Test Precision score: ",precision_score(y_test[:,0],y_pred_test,average='micro'))
        print("Test Recall score: ",recall_score(y_test[:,0],y_pred_test,average='micro'))
        print("\nConfusion Matrix test:")

        cm = confusion_matrix(y_test[:,0], y_pred_test)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
    else:
        model.fit(x_train,y_train)
        y_pred_train=model.predict(x_train)
        y_pred_test=model.predict(x_test)
        print("Train F1 score: ",f1_score(y_train,y_pred_train,average='micro'))
        print("Train Precision score: ",precision_score(y_train,y_pred_train,average='micro'))
        print("Train Recall score: ",recall_score(y_train,y_pred_train,average='micro'))
        print("Test F1 score: ",f1_score(y_test[:,0],y_pred_test,average='micro'))
        print("Test Precision score: ",precision_score(y_test[:,0],y_pred_test,average='micro'))
        print("Test Recall score: ",recall_score(y_test[:,0],y_pred_test,average='micro'))
        print("\nConfusion Matrix test:")

        cm = confusion_matrix(y_test[:,0], y_pred_test)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()


In [8]:
def regressionResults(x_train,x_test,y_train,y_test,model):
    model.fit(x_train,y_train[:,1])
    y_pred_train=model.predict(x_train)
    y_pred_test=model.predict(x_test)
    print("Train MSE: ",np.mean((y_train[:,1]-y_pred_train)**2))
    print("Train MAE: ",np.mean(np.abs(y_train[:,1]-y_pred_train)))
    print("Test MSE: ",np.mean((y_test[:,1]-y_pred_test)**2))
    print("Test MAE: ",np.mean(np.abs(y_test[:,1]-y_pred_test)))
    

In [13]:
#TODO

# >>> from sklearn import svm, datasets
# >>> from sklearn.model_selection import GridSearchCV
# >>> iris = datasets.load_iris()
# >>> parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
# >>> svc = svm.SVC()
# >>> clf = GridSearchCV(svc, parameters)
# >>> clf.fit(iris.data, iris.target)
# GridSearchCV(estimator=SVC(),
#              param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})
# >>> sorted(clf.cv_results_.keys())
# ['mean_fit_time', 'mean_score_time', 'mean_test_score',...
#  'param_C', 'param_kernel', 'params',...
#  'rank_test_score', 'split0_test_score',...
#  'split2_test_score', ...
#  'std_fit_time', 'std_score_time', 'std_test_score']

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
parameters={'n_estimators':[10,50,100,200,500],'max_depth':[5,10,20,50,100],\
            'min_samples_split':[2,5,10,20,50],'min_samples_leaf':[1,2,5,10,20],\
            'criterion':['gini','entropy',"log_loss"]}


model=RandomForestClassifier()

clf = GridSearchCV(model, parameters)
clf.fit(x_train, y_train[:,0])

print(clf.best_params_)
print(clf.best_score_)


c:\Users\Andrea\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


{'max_depth': 100, 'n_estimators': 50}
0.552


In [ ]:

from sklearn import tree
model=tree.DecisionTreeClassifier()
classificationResults(x_train,x_test,y_train,y_test,model)

#there is a lot of overfitting because decision trees try to memorize the dataset so i try to use random forest to tackle this problem 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
classificationResults(x_train,x_test,y_train,y_test,model)



In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=1000)
classificationResults(x_train,x_test,y_train,y_test,model)

In [ ]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
classificationResults(x_train,x_test,y_train,y_test,model)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
parameters={'kernel':('linear', 'poly' ,"rbf","sigmoid"), 'C':[1, 10],'gamma':["scale","auto"],'degree':[1,2,3,4,5],\
'coef0':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],'shrinking':[True,False],'probability':[True,False],'tol':[0.001,0.0001,0.00001],\
    'cache_size':[200,500,1000],'class_weight':[None,"balanced"],'verbose':[True,False],'max_iter':[-1,1000,10000,100000]}


model=svm.SVC()

clf = GridSearchCV(model, parameters)
clf.fit(x_train, y_train[:,0])

print(clf.best_params_)
print(clf.best_score_)


c:\Users\Andrea\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


KeyboardInterrupt: 

In [ ]:
from sklearn import svm
model=svm.SVC()
classificationResults(x_train,x_test,y_train,y_test,model)

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier()
classificationResults(x_train,x_test,y_train,y_test,model)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model=AdaBoostClassifier()
classificationResults(x_train,x_test,y_train,y_test,model)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier()
classificationResults(x_train,x_test,y_train,y_test,model)

In [ ]:
model=tree.DecisionTreeRegressor()
regressionResults(x_train,x_test,y_train,y_test,model)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
regressionResults(x_train,x_test,y_train,y_test,model)

In [ ]:

from sklearn.linear_model import LinearRegression
model=LinearRegression()
regressionResults(x_train,x_test,y_train,y_test,model)


In [ ]:
# import support vector regression
from sklearn.svm import SVR
model=SVR()
regressionResults(x_train,x_test,y_train,y_test,model)

In [ ]:
#import polynomial regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
model=make_pipeline(PolynomialFeatures(2),LinearRegression())
regressionResults(x_train,x_test,y_train,y_test,model)
